# Function Decorators & Dynamic Programming

## Recap
Functions are first-class objects in Python:

In [1]:
def foo(bar):
    return bar + 1

print(foo)
print(type(foo))

def caller(func, *arg):
    '''Receives an object of type function
       and calls it with the provided input argumants
    '''
    
    return func(*arg)
  
print(caller(foo, 5))

<function foo at 0x7fac239abea0>
<class 'function'>
6


Nested functions:

In [2]:
def func(bar):
    '''outer function'''
    
    print("Printing from foo()", bar)
    
    def nested_func(foo):
        '''inner function'''  
      
#         nonlocal bar

        print("Printing from the nested function", foo+bar)
#         bar = bar + 1
        
        new_bar = bar + 1
    
    nested_func(0)
    print("Printing from foo()", bar)
#     print("Printing from foo()", new_bar)
    
  
    # Returning functions from other functions
    return nested_func
    
        
func(5)
# nested_func()

new_func = func(8)
new_func(-1)

Printing from foo() 5
Printing from the nested function 5
Printing from foo() 5
Printing from foo() 8
Printing from the nested function 8
Printing from foo() 8
Printing from the nested function 7


## Decorator
Recall the Decorator design pattern which is used to dynamically add a new feature to an object without changing its implementation.

In [52]:
def my_decorator(some_function):

    def wrapper(*args, **kwargs):
        # Remember that *args and **kwargs allow 
        # us to accept a variable number of input arguments 
        # args is a tuple with all input values
        # and kwargs is a dictionary with all key = value
        # pairs provided as arguements for our function.
      
        print("Something is happening before some_function() is called.")
        
        # We need to use *args and **kwargs when calling
        # some_function so that our args tuple and
        # kwargs dictionary get re-expanded into useful
        # inputs to our function
        some_function(*args, **kwargs)
        print("Something is happening after some_function() is called.")

    return wrapper


  
def some_function(num):
    print("Printing from some_function()", num)


some_function = my_decorator(some_function)
some_function(5)

Something is happening before some_function() is called.
Printing from some_function() 5
Something is happening after some_function() is called.


Python allows you to simplify the calling of decorators using the @ symbol.

In [53]:
# @my_decorator is just an easier way of saying 
# some_function = my_decorator(some_function)
@my_decorator
def some_function():
    print("Hello world!")
    
some_function()

Something is happening before some_function() is called.
Hello world!
Something is happening after some_function() is called.


In [54]:
import time

def timer_decorator(f):
    '''
    Measures execution time of the function f
    '''
  
    def wrapper(*args, **kwargs):
        t1 = time.time()
        f(*args, **kwargs)
        t2 = time.time()
        print("Time it took to run the function: " + str((t2 - t1)))
    
    return wrapper

@timer_decorator
def some_function():
    print("Hello world!")
    
    
some_function()

Hello world!
Time it took to run the function: 0.0004584789276123047


In [4]:
import time

def sleep_decorator(f):
    '''
    Limits how fast the function f is called
    '''

    def wrapper(*args, **kwargs):
        time.sleep(2)
        return f(*args, **kwargs)
      
    return wrapper


@sleep_decorator
def print_number(num):
    return num


for num in range(1, 6):
    print(print_number(num))

1
2
3
4
5


In [6]:
def limit_chars(length):
    def decorator(function):
        def wrapper(*args, **kwargs):
            result = function(*args, **kwargs)
            return result[:length]
        return wrapper
    return decorator

@limit_chars(20) # notice parentheses
def echo(foo):
    return foo

# usage
print(echo('You cannot return more than a certain number of characters'))


def echo1(foo):
    return foo
    
dec = limit_chars(10)
echo1 = dec(echo1)
print(echo1('You cannot return more than a certain number of characters'))

You cannot return mo
You cannot


In [82]:
# the name of the example function would have been 'wrapper', 
# and the docstring of the original example() would have been lost.

print(echo.__name__)

wrapper


In [83]:
import functools

def limit_chars(length):
  
    def decorator(function):
        
        
        # functools.wraps is a decorator we can use 
        # to take all the meta data from function 
        # (like the function name and docstring) and 
        # add it to wrapper (memoized_f).
        # This way we don't lose our doctests,
        # doctests, function name, and other metadata
        @functools.wraps(function)
        def wrapper(*args, **kwargs):
            result = function(*args, **kwargs)
            result = result[:length]
        
        return wrapper
        
        
    return decorator
  
@limit_chars(10) # notice parentheses
def echo(foo):
    return foo
  
  
print(echo.__name__)

echo


For those curious, you can read more about functools at https://docs.python.org/3/library/functools.html

## Property Decorator

In [8]:
class TemperatureInCelsius:
    def __init__(self, temperature = 0):
        self.set_temperature(temperature)

    def to_fahrenheit(self):
        return (self.get_temperature() * 1.8) + 32

    # new update
    def get_temperature(self):
        return self._temperature

    def set_temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        self._temperature = value
        
        
c = TemperatureInCelsius(20)
print(c.get_temperature())
c.set_temperature(-275)
# c._temperature = -275
print(c.get_temperature())

20


ValueError: ignored

In [9]:
class TemperatureInCelsius:
    def __init__(self, temperature = 0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    def __get_temperature(self):
        print("Getting value")
        return self._temperature

    def __set_temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        print("Setting value")
        self._temperature = value

    temperature = property(__get_temperature, __set_temperature)
    
    
c = TemperatureInCelsius(20)
# This assignment automatically calls set_temperature()
c.temperature = -275
# This access automatically calls get_temperature()
print(c.temperature)

# Note that, the actual temperature value is stored 
# in the private variable _temperature. 
# The attribute temperature is a property object 
# which provides interface to this private variable.

Setting value


ValueError: ignored

In [13]:
class TemperatureInCelsius:
    def __init__(self, temperature = 0):
        self._temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    @property
    def temperature(self):
        print("Getting value")
        return self._temperature

    @temperature.setter
    def temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        print("Setting value")
        self._temperature = value
        

c = TemperatureInCelsius(20)
# This assignment automatically calls set_temperature()
c.temperature = -275
# This access automatically calls get_temperature()
print(c.temperature)

# Note that, the actual temperature value is stored 
# in the private variable _temperature. 
# The attribute temperature is a property object 
# which provides interface to this private variable.

ValueError: ignored

# Memoizing Recursive Functions

In [0]:
import functools

def memoize(f):    
    # note we're attaching the memo
    # dictionary to our function
    # here. It becomes part of the
    # data attached to our function
    # this way, if we want, we can 
    # clear our memoized values if we
    # are concerned about this dictionary 
    # growing too large
    f.memo = {}
    
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        
        my_key = repr(args) + repr(kwargs)
        # repr is like str in that it gives us a representation
        # of our object. However, str is for human reading
        # and repr is intended for "python" reading. 
        # it is guaranteed to be a unique, immutable 
        # representation of our args.
        if not my_key in f.memo:
            f.memo[my_key] = f(*args, **kwargs)
        return f.memo[my_key]
      
    return wrapper

In [14]:
# The @ symbol tells python to use memoize as a decorator. 
# This is like we did
# recursive_fib = memoize(recursive_fib)
# after declaring the function, but conveniently
# allows us to put the decorator BEFORE the function
# declaration. So anyone reading this code automatically
# knows that this code is going to be memoized.

@memoize
def recursive_fib(n):
    """
    The very slow recursive fibonacci generator. 
    Only works on postive numbers (with the 0th fibonacci number being 0)
    Time complexity: O(2^n/2)
    
    args:
    n (int): what fibonacci number you want to compute
    
    output: 
    the nth fibonacci number in sequence
    
    Errors:
    ValueError("cannot compute the fibonacci number of negative values")
    
    >>> [recursive_fib(i) for i in range(13)]
    [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]
    """
    if n<0:
        raise ValueError("cannot compute the fibonacci number of negative values")
    if n==1 or n==0:
        return n
    return recursive_fib(n-1) + recursive_fib(n-2)
  
recursive_fib(40)

102334155

When recursive_fib calls recursive_fib internally, the function checks the namespace for a variable named recursive_fib. It finds that variable, and calls it, but recalls that we've reassigned recursive_fib to be our memoized version. 

Note if we don't want to use our custom memoize function as a decorator, 
we could use the lru_cache function from functools. 

`import functools`

`@functools.lru_cache(maxsize=128, typed=False)`

This decorator wraps a function with a memoizing callable that saves up to the maxsize most recent calls. 
It can save time when an expensive or I/O bound function is periodically called with the same arguments.
Since a dictionary is used to cache results, the positional and keyword arguments to the function must be hashable.

If maxsize is set to None, the LRU feature is disabled and the cache can grow without bound. 
The LRU feature performs best when maxsize is a power-of-two. 
If typed is set to true, function arguments of different types will be cached separately. 
For example, f(3) and f(3.0) will be treated as distinct calls with distinct results.

In [99]:
import functools

@functools.lru_cache(maxsize=None, typed=False)
def recursive_fib(n):
    """
    The very slow recursive fibonacci generator. 
    Only works on postive numbers (with the 0th fibonacci number being 0)
    Time complexity: O(2^n/2)
    
    args:
    n (int): what fibonacci number you want to compute
    
    output: 
    the nth fibonacci number in sequence
    
    Errors:
    ValueError("cannot compute the fibonacci number of negative values")
    
    >>> [recursive_fib(i) for i in range(13)]
    [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]
    """
    if n<0:
        raise ValueError("cannot compute the fibonacci number of negative values")
    if n==1 or n==0:
        return n
    return recursive_fib(n-1) + recursive_fib(n-2)
  
recursive_fib(40)

102334155

In [18]:
def f(seq):
    best_value = seq[0]
    previous_value = seq[0]
    
    for j in range(1,len(seq)):
        previous_value = max(seq[j], seq[j] + previous_value)
        if(previous_value > best_value):
            best_value = previous_value
            
    return best_value
  
f([-1])

-1

In [31]:
def max_contig_sum(sequence):
    """
    The Maximum contiguous sum problem is, given a sequence, we want 
    to find the contiguous subsequence with the largest sum. 

    If our sequence is "1,2,3", then the possible contiguous subsequences
    are: "1", "1,2", "1,2,3", "2", "2,3", "3". Note that "1,3" is not a
    contiguous subsequence because 1 and 3 are not next to each other in 
    the sequence.   
      
    >>> max_contig_sum((1,2,3,-4,5,6,7,-1,8,-10,2))
    27
    >>> max_contig_sum((1,2,3,-7,5,6,7,-1,8,-10,2))
    25
    >>> max_contig_sum((1,2,3,3,-4,5,-6,7,-1,8,-10,2))
    18
    >>> max_contig_sum((1,2,3,-4,5,-6,7,-1,8,-10,2))
    15
    >>> max_contig_sum((1,2,3))
    6
    >>> max_contig_sum((1,-2,3))
    3
    >>> max_contig_sum((3,-1,-1,3))
    4
    >>> max_contig_sum((5,-1,-1,-1,4))
    6
    """
    
    max_value = 0
    
    for ending_index in range(0, len(sequence)):
        value = mcs_with_fixed_end(sequence[:ending_index+1])
        if(value > max_value):
            max_value = value
    
    return max_value
        
@memoize        
def mcs_with_fixed_end(seq):
    """
    find the maximum sum for contiguous subsequences of 
    seq ending with the last value of seq
    """
    if not seq:
        return 0
    return max(seq[-1], seq[-1] + mcs_with_fixed_end(seq[:-1]))
  
  
max_contig_sum([-1])

0

## Dynamic Programming


In [0]:
import sys
sys.setrecursionlimit(10000)

### Knapsack

We are given a collection of $n$ items. 
Item $i$ has a value $v_i \geq 0$ and a weight $w_i \geq 0$. 
We want to pack so that the value of items packed is the largest possible 
while the total weight of packed items does not exceed a maximum value, or capacity $C \geq 0$ (items are indivisible).
Formally, the goal is to find a subset of items $S\subset \{1,\dots,n\}$
with maximum possible value such that the total weight of $S$ does not exceed $C$.

In [62]:
def knapsack(k, capacity, memo=None):
    """
    Calculates the maximum value that can be 
    achieved using the items 0...k
    whose values and sizes are in the identically
    named global variables (ouch!) while
    meeting the constraint that the total size
    of packed items <=capacity.
    
    """
    
    
    if memo is None:
        memo = {}
        
    if (k,capacity) in memo:
        return memo[(k,capacity)]
      
    if k==0:
        ret = 0 if sizes[0] > capacity else values[0]
        memo[(k,capacity)] = ret
        return ret
      
    max_value = knapsack(k-1, capacity, memo)
    if sizes[k] <= capacity: # we could pack item k
        packing_value = values[k] + knapsack(k-1, capacity-sizes[k], memo)
        max_value = max(max_value, packing_value)
    memo[(k,capacity)] = max_value
    
    return max_value
  
  
values = [5,10,2,4,5]
sizes  = [10,7,3,4,1]
knapsack(len(values)-1, 10)

15

In [91]:
def bottom_up_knapsack(k, capacity):
    """
    Calculates the maximum value that can be 
    achieved using the items 0...k-1
    whose values and sizes are in the identically
    named global variables (ouch!) while
    meeting the constraint that the total size
    of packed items <=capacity.
    
    """
  
    table = [[0 for x in range(capacity+1)] for x in range(k+1)]
 
    # Build table[][] in bottom up manner
    for i in range(k+1):
        for w in range(capacity+1):
            # Base case
            if i==0 or w==0:
                table[i][w] = 0
            elif sizes[i-1] <= w:
                table[i][w] = max(values[i-1] + table[i-1][w-sizes[i-1]],  table[i-1][w])
            else:
                table[i][w] = table[i-1][w]
 
    return table[k][capacity]


values = [5,10,2,4,5]
sizes  = [10,7,3,4,1]
bottom_up_knapsack(len(values), 10)

15

### Making Change

Given a list of coin denominations and a desired amount of change, 
what is the fewest number of coins required to make this change?

In [98]:
import math

def make_change(value, memo=None):
    """
    The fewest number of coins required to make a desired amount of change
    given a list of coin denominations
    
    
    >>> denoms = [1, 8, 12]
    >>> make_change(20)
    2
    >>> make_change(16)
    2
    >>> make_change(15)
    4
    >>> make_change(6)
    6
    >>> make_change(0)
    0
    >>> make_change(9)
    2
    >>> make_change(100)
    9
    """
    if memo is None:
        memo = {}
    
    if value < 0:
        return math.inf
    
    if not value in memo:
        if value == 0:
            memo[value] = 0
        else:
            memo[value] = 1 + min([make_change(value - denoms[i], memo) 
                                   for i in range(len(denoms))])    
        
    return memo[value]
  

denoms = [1, 8, 12]
make_change(200)

17

In [96]:
def bottom_up_make_change(value):
    table = [0 for amt in range(value+1)]
 
    # Fill the rest of the table entries in bottom up manner
    for i in range(1, value+1):
        table[i] = math.inf
        for j in range(len(denoms)):
            if denoms[j] <= i and 1 + table[i - denoms[j]] < table[i]:
                table[i] = 1 + table[i - denoms[j]]
 
    return table[value]


denoms = [1, 8, 12]
denoms = sorted(denoms)
bottom_up_make_change(200)

9

### Optimal Parenthesization for Matrix Chain Product

Given a chain of N matrices $A_1, A_2, \cdots, A_N$, 
where for $i = \{1,2, \cdots , N\}$ matrix $A_i$ has dimension $d_{i-l} \times d_i$, 
fully parenthesize the product $A_1 \times A_2 \times \cdots \times A_n$
in a way that minimizes the number of scalar multiplications required to evaluate this expression. 

Note that evaluating $A_1 \times A_2$ 
requires $d_0 \times d_1 \times d_2$ scalar multiplications.

In [8]:
def opt_product(i, j, memo=None):
    if memo is None:
        memo = {}
    if (i, j) in memo:
        return memo[(i,j)]
    if j == i+1:
        cost = 0
    else:
        cost = min([opt_product(i, k, memo) + opt_product(k, j, memo) \
                    + d[i-1]*d[j-1]*d[k-1] for k in range(i+1, j)])
    memo[(i,j)] = cost
    return cost

  
d = [10, 100, 5, 50]  
opt_product(1, len(d))

7500

In [54]:
import math

def bottom_up_opt_product():
    # cost[m][n] is the cost of evaluating A_m x ... x A_{n-1}
    cost = [[math.inf for i in range(len(d))] for j in range(len(d))]
    
    for k in range(1, len(d)+1):
        for i in range(len(d)-k):
            if k == 1:
                cost[i][i+k] = 0
                continue
            for j in range(i+1, i+k):
                current_cost = cost[i][j] + cost[j][i+k] + d[i]*d[j]*d[i+k]
                if current_cost < cost[i][i+k]:
                    cost[i][i+k] = current_cost
                  
    return cost[0][len(d)-1]

  
d = [10, 100, 5, 50]  
bottom_up_opt_product()

7500